In [2]:
# filename: Dissertation_Airline_Delays_core.py
# Inputs: flights.csv, airlines.csv, airports.csv  [Kaggle "2015 Flight Delays & Cancellations"]
# Outputs:
# Data_clean -> flights_2015_ATL-LAX_JFK-ORD.csv
# Outputs -> (KPIs, cause mix, schedule padding, time series)
# Outputs -> (Figures used in the Dissertation)

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import sqrt
from scipy.stats import norm
# ---------------------------
# 0) Paths & setup
# ---------------------------
FLIGHTS_PATH  = "flights.csv"
AIRLINES_PATH = "airlines.csv"
AIRPORTS_PATH = "airports.csv"

OUT_DATA_DIR = "Data_clean"
OUT_DIR      = "Outputs"
os.makedirs(OUT_DATA_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

# ---------------------------
# 1) Read and filter data efficiently
# ---------------------------
COLUMNS = ["YEAR","MONTH","DAY","AIRLINE","FLIGHT_NUMBER",
           "ORIGIN_AIRPORT","DESTINATION_AIRPORT",
           "SCHEDULED_DEPARTURE","DEPARTURE_TIME","DEPARTURE_DELAY",
           "SCHEDULED_ARRIVAL","ARRIVAL_TIME","ARRIVAL_DELAY",
           "SCHEDULED_TIME","ELAPSED_TIME","DISTANCE",
           "DIVERTED","CANCELLED",
           "AIR_SYSTEM_DELAY","SECURITY_DELAY","AIRLINE_DELAY","LATE_AIRCRAFT_DELAY","WEATHER_DELAY"]

routes = {"ATL", "LAX", "JFK", "ORD"}
pairs = {("ATL", "LAX"), ("LAX", "ATL"), ("JFK", "ORD"), ("ORD", "JFK")}

filtered_part = []
for chunk in pd.read_csv(FLIGHTS_PATH, usecols=COLUMNS, chunksize=1_000_000, low_memory=False):
    a = chunk["ORIGIN_AIRPORT"].isin(routes) & chunk["DESTINATION_AIRPORT"].isin(routes)
    if not a.any(): 
        continue
    
    c = chunk[a]
    b = list(zip(c["ORIGIN_AIRPORT"], c["DESTINATION_AIRPORT"]))
    c = c[[t in pairs for t in b]]
    c = c[c["YEAR"] == 2015]
    
    if len(c):
        filtered_part.append(c)

p = pd.concat(filtered_part, ignore_index=True)

# ---------------------------
# 2) Feature Engineering
# ---------------------------
p["FlightDate"] = pd.to_datetime(p[["YEAR", "MONTH", "DAY"]])
p["Month"] = p["FlightDate"].dt.month
p["RoutePair"] = np.where(p["ORIGIN_AIRPORT"].isin(["ATL", "LAX"]), "ATL-LAX", "JFK-ORD")
p["Direction"] = p["ORIGIN_AIRPORT"] + "→" + p["DESTINATION_AIRPORT"]
p["Cancelled"] = p["CANCELLED"].fillna(0).astype(int)
p["Diverted"]  = p["DIVERTED"].fillna(0).astype(int)

# Vectorized hour from HHMM (handles NaN)
dep_num = pd.to_numeric(p["SCHEDULED_DEPARTURE"], errors="coerce")
p["DepHourLocal"] = (dep_num // 100).astype("Int16")

# Create Delayed15 flag (only for operated flights, NaN otherwise)
mask_op = (p["Cancelled"] == 0) & (p["Diverted"] == 0)
p.loc[mask_op, "Delayed_by_15 min"] = (p.loc[mask_op, "ARRIVAL_DELAY"] >= 15).astype(int)

operated = p[mask_op].copy()


# ---------------------------
# 3) Save the cleaned subset used in the report
# ---------------------------
filtered_cols = [
    "FlightDate","AIRLINE","FLIGHT_NUMBER","ORIGIN_AIRPORT","DESTINATION_AIRPORT",
    "SCHEDULED_DEPARTURE","DEPARTURE_TIME","DEPARTURE_DELAY",
    "SCHEDULED_ARRIVAL","ARRIVAL_TIME","ARRIVAL_DELAY",
    "SCHEDULED_TIME","ELAPSED_TIME","DISTANCE",
    "Cancelled","Diverted","RoutePair","Direction","Month","DepHourLocal","Delayed_by_15 min",
    "AIR_SYSTEM_DELAY","SECURITY_DELAY","AIRLINE_DELAY","LATE_AIRCRAFT_DELAY","WEATHER_DELAY"
]
filtered = p[filtered_cols].copy()
filtered_path = os.path.join(OUT_DATA_DIR, "flights_2015_ATL-LAX_JFK-ORD.csv")
filtered.to_csv(filtered_path, index=False)
print(f"Filtered Dataset Saved: {filtered_path}")

# ---------------------------
# 4) KPIs (per route pair, calendar year 2015)
# ---------------------------
def kpis_by_routepair(all, op):
    rows = []
    for rp, g_sched in all.groupby("RoutePair"):
        scheduled = len(g_sched)
        canc = int(g_sched["Cancelled"].sum())
        div  = int(g_sched["Diverted"].sum())

        g = op[op["RoutePair"] == rp]
        operated_n = len(g)

        if operated_n:
            pct_ge15  = 100.0 * g["ARRIVAL_DELAY"].ge(15).mean()   # % flights >=15 min late
            pct_ge60  = 100.0 * g["ARRIVAL_DELAY"].ge(60).mean()   # % flights >=60 min late
            ontime_pct = 100.0 - pct_ge15
            avg_delay  = g["ARRIVAL_DELAY"].mean()
        else:
            pct_ge15 = pct_ge60 = ontime_pct = avg_delay = np.nan

        rows.append(dict(
            RoutePair=rp,
            flights_scheduled=scheduled,
            flights_operated=operated_n,
            cancellations=canc,
            cancellation_rate=100.0 * canc / scheduled if scheduled else np.nan,
            pct_delay_ge15=pct_ge15,              
            pct_delay_ge60=pct_ge60,              
            ontime_pct=ontime_pct,
            avg_arrival_delay_min=avg_delay
        ))
    return pd.DataFrame(rows)

kpi = kpis_by_routepair(p, operated)
def add_otp_confints(df, operated_df):
    rows = []
    for rp, g in operated_df.groupby("RoutePair"):
        n = len(g)
        if n == 0:
            continue
        p_hat = (g["ARRIVAL_DELAY"] < 15).mean()
        # Normal approx 95% CI
        z = 1.96
        se = sqrt(p_hat * (1 - p_hat) / n)
        lower = max(0, p_hat - z * se)
        upper = min(1, p_hat + z * se)
        rows.append({"RoutePair": rp,
                     "n_operated": n,
                     "otp_prop": p_hat,
                     "otp_ci_lower": lower,
                     "otp_ci_upper": upper})
    return pd.DataFrame(rows)

otp_CI = add_otp_confints(kpi, operated)
otp_CI_path = os.path.join(OUT_DIR, "otp_confidence_intervals.csv")
otp_CI.to_csv(otp_CI_path, index=False)
print(f"On-time confidence intervals saved: {otp_CI_path}")
kpi_path = os.path.join(OUT_DIR, "kpi_by_routepair.csv")
kpi.to_csv(kpi_path, index=False)
print(f"KPI table saved: {kpi_path}")

stable_months = [3, 4, 5, 8, 9, 10, 11, 12]

p_stable = p[p["Month"].isin(stable_months)].copy()
operated_stable = operated[operated["Month"].isin(stable_months)].copy()

kpi_stable = kpis_by_routepair(p_stable, operated_stable)
kpi_stable_path = os.path.join(OUT_DIR, "kpi_by_routepair_stable_months.csv")
kpi_stable.to_csv(kpi_stable_path, index=False)
print(f"KPI for stable months table saved: {kpi_stable_path}")


# ---------------------------
# 5) Monthly on-time % (per route pair)
# ---------------------------
monthly = (operated.groupby(["RoutePair","Month"])["Delayed_by_15 min"].apply(lambda s: 100*(1 - s.mean())).reset_index(name="OnTimePct"))
monthly_path = os.path.join(OUT_DIR, "monthly_otp_by_routepair.csv")
monthly.to_csv(monthly_path, index=False)

plt.figure(figsize=(7,4.5))
for rp, g in monthly.groupby("RoutePair"):
    k = g.sort_values("Month")
    plt.plot(k["Month"], k["OnTimePct"], marker="o", label=rp)
plt.title("Monthly On-Time % (Arrival < 15 min late) 2015")
plt.xlabel("Month"); plt.ylabel("On-Time Percentage"); plt.ylim(0,100); plt.grid(True, alpha=0.3)
plt.xticks(range(1,13)); plt.legend()
plt.tight_layout(); plt.savefig(os.path.join(OUT_DIR, "Monthly_On-time_Performance.png"), dpi=200); plt.close()

# ---------------------------
# 6) Hour-of-day on-time % (per route pair)
# ---------------------------
hod = (operated[operated["DepHourLocal"].notna()].groupby(["RoutePair","DepHourLocal"])["Delayed_by_15 min"].apply(lambda s: 100*(1 - s.mean())).reset_index(name="OnTimePct"))
hod_path = os.path.join(OUT_DIR, "Hour_of_Day_OTP_by_routepair.csv")
hod.to_csv(hod_path, index=False)

plt.figure(figsize=(7,4.5))
for rp, g in hod.groupby("RoutePair"):
    k = g.sort_values("DepHourLocal")
    plt.plot(k["DepHourLocal"], k["OnTimePct"], marker="o", label=rp)
plt.title("On-Time % by Scheduled Departure Hour 2015")
plt.xlabel("Scheduled Departure Hour (local)"); plt.ylabel("On-Time Percentage"); plt.ylim(0,100)
plt.grid(True, alpha=0.3); plt.legend(); plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "Hour_of_Day_On-time_Performance.png"), dpi=200); plt.close()

# ---------------------------
# 7) Delay distribution (histograms) per route pair
# ---------------------------
for rp in operated["RoutePair"].unique():
    x = operated.loc[operated["RoutePair"]==rp, "ARRIVAL_DELAY"].clip(lower=-30, upper=180)
    plt.figure(figsize=(7,4.5))
    plt.hist(x.dropna(), bins=40)
    plt.title(f"Arrival Delay Distribution — {rp} (2015)")
    plt.xlabel("Arrival Delay (minutes)"); plt.ylabel("Flights")
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, f"delay_hist_{rp.replace('-','_')}.png"), dpi=200)
    plt.close()

# ---------------------------
# 8) Delay cause mix (minutes) per route pair
#    (AIRLINE_DELAY = carrier; WEATHER_DELAY = weather; AIR_SYSTEM_DELAY = NAS; SECURITY_DELAY; LATE_AIRCRAFT_DELAY)
# ---------------------------
cause_cols = ["AIRLINE_DELAY","WEATHER_DELAY","AIR_SYSTEM_DELAY","SECURITY_DELAY","LATE_AIRCRAFT_DELAY"]
cause = (operated.groupby("RoutePair")[cause_cols].sum(min_count=1).reset_index())
cause.to_csv(os.path.join(OUT_DIR, "delay_cause_minutes_by_routepair.csv"), index=False)

# Shares (Percentage of total delay minutes reported)
Shares = cause.copy()
Total = Shares[cause_cols].sum(axis=1)
for w in cause_cols:
    Shares[w + "_SHARE"] = (Shares[w] / Total) * 100
share_cols = ["RoutePair"] + [w + "_SHARE" for w in cause_cols]
Shares[share_cols].to_csv(os.path.join(OUT_DIR, "delay_cause_shares_by_routepair.csv"), index=False)

# Optional stacked bar
plot = cause.set_index("RoutePair")[cause_cols].fillna(0)
if plot.to_numpy().sum() > 0:
    plot.plot(kind="bar", stacked=True, figsize=(7,4.5))
    plt.ylabel("Delay Minutes (sum)"); plt.title("Delay Cause Minutes 2015")
    plt.tight_layout(); plt.savefig(os.path.join(OUT_DIR, "Delay_Cause_Stacked.png"), dpi=200); plt.close()

# ---------------------------
# 9) Schedule padding (block-time slack)
#    padding = median(SCHEDULED_TIME) − median(ELAPSED_TIME among on-time flights)
#    computed by Direction (ATL→LAX, LAX→ATL, JFK→ORD, ORD→JFK) and by Month
# ---------------------------

# "Unimpeded" actual = median ELAPSED_TIME among on-time flights (ARRIVAL_DELAY < 15)
unimpeded = (operated[operated["Delayed_by_15 min"]==0].groupby("Direction")["ELAPSED_TIME"].median().rename("UnimpededMedian_ACT_minutes").reset_index())

scheduled = (operated.groupby("Direction")["SCHEDULED_TIME"].median().rename("Median_Scheduled_minutes").reset_index())

padding = pd.merge(scheduled, unimpeded, on="Direction", how="left")
padding["Padding_minutes"] = padding["Median_Scheduled_minutes"] - padding["UnimpededMedian_ACT_minutes"]
padding_direction = padding.sort_values("Direction")
padding_path = os.path.join(OUT_DIR, "schedule_padding_by_direction.csv")
padding_direction.to_csv(padding_path, index=False)

plt.figure(figsize=(8,4.8))
x = np.arange(len(padding_direction))
plt.bar(x, padding_direction["Padding_minutes"])
plt.xticks(x, padding_direction["Direction"], rotation=20)
plt.ylabel("Padding (minutes)")
plt.title("Schedule Padding by Direction (Median Scheduled - Median Unimpeded Actual)")
plt.tight_layout(); plt.savefig(os.path.join(OUT_DIR, "Padding_by_Direction.png"), dpi=200); plt.close()

# Monthly padding by direction (optional but useful for Appendix)
unimpeded_monthly = (operated[operated["Delayed_by_15 min"]==0].groupby(["Direction","Month"])["ELAPSED_TIME"].median().rename("Unimpeded_median").reset_index())
scheduled_monthly = (operated.groupby(["Direction","Month"])["SCHEDULED_TIME"].median().rename("Scheduled_median").reset_index())
pm = pd.merge(scheduled_monthly, unimpeded_monthly, on=["Direction","Month"], how="left")
pm["Padding_minutes"] = pm["Scheduled_median"] - pm["Unimpeded_median"]
pm.to_csv(os.path.join(OUT_DIR, "Schedule_Padding_Monthly_by_Direction.csv"), index=False)

for d in sorted(pm["Direction"].unique()):
    a = pm[pm["Direction"]==d].sort_values("Month")
    plt.figure(figsize=(7,4.5))
    plt.plot(a["Month"], a["Padding_minutes"], marker="o")
    plt.title(f"Monthly Schedule Padding — {d} (2015)")
    plt.xlabel("Month"); plt.ylabel("Padding (minutes)"); plt.grid(True, alpha=0.3)
    plt.xticks(range(1,13)); plt.tight_layout()
    fn = f"padding_monthly_{d.replace('→','to')}.png"
    plt.savefig(os.path.join(OUT_DIR, fn), dpi=200); plt.close()

print(" Cleaned file:", filtered_path)
print(" Tables:      ./Outputs/*.csv")
print(" Figures:     ./Outputs/*.png")


Filtered Dataset Saved: Data_clean\flights_2015_ATL-LAX_JFK-ORD.csv
On-time confidence intervals saved: Outputs\otp_confidence_intervals.csv
KPI table saved: Outputs\kpi_by_routepair.csv
KPI for stable months table saved: Outputs\kpi_by_routepair_stable_months.csv
 Cleaned file: Data_clean\flights_2015_ATL-LAX_JFK-ORD.csv
 Tables:      ./Outputs/*.csv
 Figures:     ./Outputs/*.png
